In [44]:
import pandas as pd

In [64]:
# df = pd.read_csv('/media/walml/mirror/galaxy_zoo/older_projects/weekly_dumps/2017-10-15_galaxy_zoo_ukidss_classifications.csv')
# image_root = '/home/walml/repos/galaxy-datasets/derived_data/ukidss'




# nope, there rest are not here - though possibly they are simply on a different S3 subfolder. I can check with Zach.



# df = pd.read_csv('/media/walml/mirror/galaxy_zoo/older_projects/weekly_dumps/2017-10-15_galaxy_zoo_illustris_classifications.csv')
# image_root = '/home/walml/repos/galaxy-datasets/derived_data/illustris'

# df = pd.read_csv('/media/walml/mirror/galaxy_zoo/older_projects/weekly_dumps/2017-10-15_galaxy_zoo_gama09_classifications.csv')
# image_root = '/home/walml/repos/galaxy-datasets/derived_data/gama09'

df = pd.read_csv('/media/walml/mirror/galaxy_zoo/older_projects/weekly_dumps/2017-10-22_galaxy_zoo_gama12_classifications.csv')
image_root = '/home/walml/repos/galaxy-datasets/derived_data/gama12'

In [65]:
df.sample()

,id,subject_id,user,created_at,lang,user_agent,gama-0,gama-1,gama-2,gama-3,gama-4,gama-5,gama-6,gama-7,gama-8,gama-9,gama-10,gama-11
17370,595b8ad361ec367412000991,5857e790d369fd00400061ed,MauiWaui,2017-07-04 12:32:19 UTC,de,unknown,a-1,a-1,a-1,a-1,a-0,NaN,NaN,NaN,NaN,a-3,a-0,a-1


In [58]:
subject_df = df[['subject_id']].drop_duplicates()
len(subject_df)

11507

In [48]:
subject_df

,subject_id
0,5857c531d369fd0040005911
1,5857c532d369fd00400059df
2,5857c4a8d369fd0040000377
3,5857c4a5d369fd0040000125
4,5857c4a7d369fd004000023d
...,...
167178,5857c515d369fd00400043fb
169795,5857c4dcd369fd0040001abf
182242,5857c4e3d369fd0040001f73
183611,5857c4e0d369fd0040001d55


In [49]:

subject_df['subfolder'] = subject_df['subject_id'].apply(lambda x: x[:21])
# subject_df['file_exists']
# subject_df['subfolder'].value_counts()

In [50]:
subject_df['url'] = 'https://s3.amazonaws.com/www.galaxyzoo.org/subjects/standard/' + subject_df['subject_id'] + '.jpg'
subject_df['url'].iloc[0]

'https://s3.amazonaws.com/www.galaxyzoo.org/subjects/standard/5857c531d369fd0040005911.jpg'

In [51]:
# subject_df['file_loc'] = image_root + '/' + subject_df['subfolder'] + '/' + subject_df['subject_id'] + '.jpg'
# subject_df['file_loc'].iloc[0]

In [52]:
# pull all of these with wget
with open(image_root + '/latest_image_urls.txt', 'w') as f:
    # missing files only
    # f.write('\n'.join(df[~df['file_exists']]['url'].values))
    f.write('\n'.join(subject_df['url'].values))

In [53]:
# cat latest_image_urls.txt  | xargs -n10 -P4 wget --continue --quiet